In [1]:
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
from numpy.lib import stride_tricks

from sklearn.model_selection import train_test_split

from scipy import signal
from scipy.io import wavfile as wav

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

#import cv2
import librosa
import librosa.display

from tensorflow.keras import layers, models
import tensorflow as tf
from sklearn.metrics import accuracy_score

# W&B
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger

import time

In [7]:
import os
import subprocess

# Define the GitHub repository URL
github_repo_url = "https://github.com/gveres/donateacry-corpus.git"

# Clone the repository
subprocess.run(["git", "clone", github_repo_url])

# Define the path to the cloned repository
repo_path = "donateacry-corpus"

# Initialize empty lists to store WAV file paths and folder names
wav_files = []
folder_names = []

# Walk through the directory structure to find WAV files and their folders
for root, dirs, files in os.walk(repo_path):
    for file in files:
        if file.endswith(".wav"):
            # Get the full path of the WAV file
            wav_path = os.path.join(root, file)

            # Extract the folder name from the path
            folder_name = os.path.basename(root)

            # Append the WAV file path and folder name to the respective lists
            wav_files.append(wav_path)
            folder_names.append(folder_name)

In [8]:
def wav_to_spectrogram(file_path, output_shape=(128, 64)):
    # Load the WAV file
    y, sr = librosa.load(file_path, sr=None)
    # print(sr)

    # Generate the spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

    # Resize the spectrogram to the desired shape (e.g., 128x64)
    spectrogram = librosa.util.fix_length(spectrogram, size=output_shape[1], mode='constant', constant_values=0)

    # Normalize the spectrogram to values between 0 and 1
    spectrogram = librosa.util.normalize(spectrogram)

    return spectrogram, sr

# Example usage:
# spectrogram, sr = wav_to_spectrogram('donateacry_corpus_cleaned_and_updated_data/belly_pain/69BDA5D6-0276-4462-9BF7-951799563728-1436936185-1.1-m-26-bp.wav')

In [10]:
# Initialize empty lists to store spectrograms and labels
X = []
y = []

# Iterate through the WAV files and folder names
for wav_file, folder_name in zip(wav_files, folder_names):
    # Check if the folder name is one of the labels
    if folder_name in labels:
        label_idx = labels.index(folder_name)

        # Convert WAV to spectrogram using the wav_to_spectrogram function
        spectrogram, sr = wav_to_spectrogram(wav_file)

        # Append the spectrogram to X
        X.append(spectrogram)

        # Append the label index (numeric representation of the label) to y
        y.append(label_idx)

# Convert lists to NumPy arrays for further processing
X = np.array(X)
y = np.array(y)

# Check the shapes
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

X shape: (457, 128, 64)
y shape: (457,)


In [11]:
y

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=215)

In [13]:
num_classes = len(np.unique(y_train))

model = models.Sequential([
    layers.Input(shape=(128, 64)),  # Adjust the input shape based on your spectrogram size
    layers.Reshape(target_shape=(128, 64, 1)),  # Add a channel dimension
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [14]:
wandb.login()

wandb: Currently logged in as: jessica-gochioco (ac215-ccb). Use `wandb login --relogin` to force relogin


True

In [17]:
# Convert labels to one-hot encoding (assuming you have more than one class)
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes)

# Train the model
batch_size = 32
epochs = 10
model_name = 'baseline'

# Initialize a W&B run
wandb.init(
    project = 'crycrybaby',
    config = {
      "epochs": epochs,
      "batch_size": batch_size,
      "model_name": model_name
    },
    )

# Train model
start_time = time.time()

training_results = model.fit(
            X_train,
            y_train_encoded,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.1,
            callbacks=[WandbCallback()],)

execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)

# Update W&B
wandb.config.update({"execution_time": execution_time})
# Close the W&B run
wandb.run.finish()

# history = model.fit(X_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
11/11 [==============================] - ETA: 0s - loss: 1.0959 - accuracy: 0.7652

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231007_223533-g8oktw5p/files/model-best)... Done. 0.9s


11/11 [==============================] - 6s 489ms/step - loss: 1.0959 - accuracy: 0.7652 - val_loss: 0.8758 - val_accuracy: 0.7838
Epoch 2/10
11/11 [==============================] - 10s 844ms/step - loss: 0.4900 - accuracy: 0.8537 - val_loss: 0.9661 - val_accuracy: 0.7838
Epoch 3/10
11/11 [==============================] - 3s 255ms/step - loss: 0.3077 - accuracy: 0.8933 - val_loss: 0.8845 - val_accuracy: 0.7838
Epoch 4/10
11/11 [==============================] - 2s 199ms/step - loss: 0.1680 - accuracy: 0.9573 - val_loss: 1.0493 - val_accuracy: 0.7838
Epoch 5/10
11/11 [==============================] - 3s 240ms/step - loss: 0.0804 - accuracy: 0.9939 - val_loss: 1.1612 - val_accuracy: 0.7838
Epoch 6/10
11/11 [==============================] - 4s 334ms/step - loss: 0.0369 - accuracy: 1.0000 - val_loss: 1.2447 - val_accuracy: 0.7838
Epoch 7/10
11/11 [==============================] - 4s 313ms/step - loss: 0.0186 - accuracy: 1.0000 - val_loss: 1.4038 - val_accuracy: 0.7838
Epoch 8/10
11/11

accuracy,▁▄▅▇██████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,▁▂▁▃▄▄▆▆▇█
accuracy,1.0
best_epoch,0
best_val_loss,0.87581
epoch,9
loss,0.00506
val_accuracy,0.78378


In [18]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_encoded, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'Test accuracy: {accuracy:.2f}')

3/3 [==============================] - 0s 47ms/step
Test accuracy: 0.84


In [19]:
len(y_pred_classes)

92

In [20]:
y_pred_classes

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3])

In [21]:
np.full(len(y_pred_classes), 3)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3])

In [22]:
hungry_test = np.full(len(y_pred_classes), 3)

In [23]:
accuracy_score(y_test_classes, hungry_test)

0.8369565217391305

In [24]:

## let's try another model

num_classes = len(np.unique(y_train))


vggish_model = models.Sequential([
    layers.Input(shape=(128, 64)),  # Adjust the input shape based on your spectrogram size
    layers.Reshape(target_shape=(128, 64, 1)),  # Add a channel dimension
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), strides=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), strides=(2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), strides=(2, 2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
vggish_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [25]:
# Convert labels to one-hot encoding (assuming you have more than one class)
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes)

# Train the model
batch_size = 32
epochs = 10
model_name = 'vgg-ish'

# Initialize a W&B run
wandb.init(
    project = 'crycrybaby',
    config = {
      "epochs": epochs,
      "batch_size": batch_size,
      "model_name": model_name
    },
    )

# Train model
start_time = time.time()

training_results = vggish_model.fit(
            X_train,
            y_train_encoded,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.1,
            callbacks=[WandbCallback()],)

execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)

# Update W&B
wandb.config.update({"execution_time": execution_time})
# Close the W&B run
wandb.run.finish()

# history = model.fit(X_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
11/11 [==============================] - ETA: 0s - loss: 1.2551 - accuracy: 0.7652 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231007_224504-lo3v9h6y/files/model-best)... Done. 15.2s


11/11 [==============================] - 171s 16s/step - loss: 1.2551 - accuracy: 0.7652 - val_loss: 0.9195 - val_accuracy: 0.7838
Epoch 2/10
11/11 [==============================] - ETA: 0s - loss: 0.7410 - accuracy: 0.8415 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231007_224504-lo3v9h6y/files/model-best)... Done. 10.8s


11/11 [==============================] - 150s 14s/step - loss: 0.7410 - accuracy: 0.8415 - val_loss: 0.8516 - val_accuracy: 0.7838
Epoch 3/10
11/11 [==============================] - 116s 10s/step - loss: 0.6614 - accuracy: 0.8415 - val_loss: 0.8669 - val_accuracy: 0.7838
Epoch 4/10
11/11 [==============================] - 106s 10s/step - loss: 0.6756 - accuracy: 0.8415 - val_loss: 0.8859 - val_accuracy: 0.7838
Epoch 5/10
11/11 [==============================] - 111s 10s/step - loss: 0.6647 - accuracy: 0.8415 - val_loss: 0.8772 - val_accuracy: 0.7838
Epoch 6/10
11/11 [==============================] - ETA: 0s - loss: 0.6851 - accuracy: 0.8415 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231007_224504-lo3v9h6y/files/model-best)... Done. 11.3s


11/11 [==============================] - 150s 13s/step - loss: 0.6851 - accuracy: 0.8415 - val_loss: 0.8379 - val_accuracy: 0.7838
Epoch 7/10
11/11 [==============================] - ETA: 0s - loss: 0.6661 - accuracy: 0.8415 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231007_224504-lo3v9h6y/files/model-best)... Done. 10.6s


11/11 [==============================] - 162s 15s/step - loss: 0.6661 - accuracy: 0.8415 - val_loss: 0.8117 - val_accuracy: 0.7838
Epoch 8/10
11/11 [==============================] - 117s 10s/step - loss: 0.6538 - accuracy: 0.8415 - val_loss: 0.8251 - val_accuracy: 0.7838
Epoch 9/10
11/11 [==============================] - 107s 10s/step - loss: 0.6540 - accuracy: 0.8415 - val_loss: 0.8214 - val_accuracy: 0.7838
Epoch 10/10
11/11 [==============================] - 120s 11s/step - loss: 0.6628 - accuracy: 0.8415 - val_loss: 0.8347 - val_accuracy: 0.7838
Training execution time (mins) 21.853917360305786


accuracy,▁█████████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▂▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▅▆▅▃▁▂▂▂
accuracy,0.84146
best_epoch,6
best_val_loss,0.81166
epoch,9
loss,0.66283
val_accuracy,0.78378


In [26]:
# Evaluate the model on the test set
y_pred = vggish_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_encoded, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'Test accuracy: {accuracy:.2f}')

3/3 [==============================] - 12s 3s/step
Test accuracy: 0.84
